In [1]:
# !pip uninstall pyphi -y
# !git clone -b feature/iit-4.0 --single-branch https://github.com/wmayner/pyphi.git
# !pip install -e pyphi

In [2]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

In [3]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:80% !important; }</style>"))

In [4]:
from visualize_pyphi import *
from visualize_pyphi import utils
from visualize_pyphi import compute
import pickle as pkl
from tqdm.auto import tqdm
import numpy as np

# 4 nodes linear

In [5]:
# from scipy.stats import lognorm
# import numpy as np
# import pylab as pl

# stddev = 0.6
# mean = 0.3
# dist=lognorm([stddev],loc=mean)
# x=np.linspace(0,4,100)
# pl.plot(x,dist.pdf(x))
# [dist.pdf(x) for x in [1,2,3,4]]

In [6]:
# stddev = 0.59
# mean = 0.3
# dist=lognorm([stddev],loc=mean)
# x=np.linspace(0,4,100)
# pl.plot(x,dist.pdf(x))
# [dist.pdf(x) for x in [1,2,3,4]]

In [7]:
import pickle
def mice_to_int(ces, mices):
    return [ces.index(mice) for mice in mices]

def indirect_part(ces, part):
    return [mice_to_int(ces, (part.relata[i] for i in part.mechanism)), sorted(part.purview)]

def indirect_partition(ces, partition):
    return [indirect_part(ces, part) for part in partition]

def indirect_relation(ces, relation):
    return [mice_to_int(ces, relation.relata), relation.purview, relation.phi, indirect_partition(ces, relation.partition)]

def compress_relations(ces, relations, to_pickle=''):
    compressed_relations = [indirect_relation(ces, rel) for rel in relations]
    if to_pickle:
        with open(to_pickle, "wb") as f:
            pickle.dump(compressed_relations, f)
#     return compressed_relations

In [8]:
# ch3 system creation
net_name = "space_4"

th = 1 / 4
exp = 5

s = .8
l = 0.26
f = 0.06
g = 0.0

weights = np.array(
    [
        [s, l, f, g],  # A
        [l, s, l, f],  # B
        [f, l, s, l],  # C
        [g, f, l, s],  # D
        #A, B, C, D,
    ]
)

# weights += np.random.random((6,6))/100
# weights = np.abs(weights)

node_labels = ["A", "B", "C", "D"]
mech_func = [f for f in "ssss"]

network = ng.get_net(
    mech_func,
    weights,
    exp=exp,
    th=th,
    node_labels=node_labels,
    network_name=net_name,
    pickle_network=True,
)

state = (0, 0, 0, 0)
subsystem = pyphi.Subsystem(network, state)

  0%|          | 0/16 [00:00<?, ?it/s]

Network saved to: space_4


In [9]:
ces = utils.flatten([[d.cause,d.effect] for d in pyphi.compute.ces(subsystem,parallel=True)])

In [49]:
# all_compositional_states = compute.get_all_compositional_states(ces)
# print('there are {} possible compositional states'.format(len(all_compositional_states)))

In [ ]:
for k in tqdm(range(15,len(ces))):
    print(f'Computing k = {k}...')
    krels = list(pyphi.relations.relations(subsystem,ces,min_order=k,max_order=k,parallel=True,parallel_kwargs=dict(n_jobs=120,batch_size=10000,verbose=5)))
    compress_relations(ces, krels, f'pickles/{net_name}_rels{k}_compressed.pkl')

  0%|          | 0/15 [00:00<?, ?it/s]

Computing k = 15...


[Parallel(n_jobs=120)]: Using backend LokyBackend with 120 concurrent workers.
[Parallel(n_jobs=120)]: Done  84 tasks      | elapsed:  2.1min
[Parallel(n_jobs=120)]: Done 4560 tasks      | elapsed: 47.3min
[Parallel(n_jobs=120)]: Done 19140 tasks      | elapsed: 100.5min
[Parallel(n_jobs=120)]: Done 36960 tasks      | elapsed: 189.6min


In [ ]:
# %%time
# krels = list(utils.parallcompute_ks_relations(subsystem,ces,ks=[k],n_jobs=120,chunk_size=10000,verbose=5))
# compress_relations(ces, krels, f'pickles/{net_name}_rels{k}_compressed_test_mine.pkl')

In [ ]:
# %%time
# krels = list(pyphi.relations.relations(subsystem,ces,min_order=k,max_order=k,parallel=True,parallel_kwargs=dict(n_jobs=120,batch_size=10000,verbose=5)))
# compress_relations(ces, krels, f'pickles/{net_name}_rels{k}_compressed_test_pyphi.pkl')

In [ ]:
# k=4
# rels = list(pyphi.relations.relations(subsystem,ces,parallel=True,min_order=k,max_order=k,parallel_kwargs=dict(verbose=5,n_jobs=-1)))

In [ ]:
compressed_rs = compress_relations(ces, rels, f'pickles/{net_name}_rels{k}_compressed.pkl')

In [ ]:

compress_relations(ces, rels, f'pickles/{net_name}_rels{k}_compressed.pkl')

In [ ]:
relations = pyphi.relations.relations(subsystem,ces,parallel=True,max_order=3,parallel_kwargs=dict(verbose=5,n_jobs=-1))

max_ces = max(
    [
        compute.filter_ces(subsystem, ces, compositional_state, max_relations_k=3, relations=relations)
        for compositional_state in tqdm(all_compositional_states)
    ],
    key=lambda c: c[0]["big phi"],
)

In [ ]:
# max_ces2 = max(
#     [
#         compute.filter_ces(subsystem, ces, compositional_state, max_relations_k=3, relations=[])
#         for compositional_state in tqdm(all_compositional_states)
#     ],
    
#     key=lambda c: c[0]["big phi"],
# )

In [ ]:
utils.sepces2df(max_ces[0]['ces'],subsystem)

In [ ]:
max_ces[0]['MIP']

In [ ]:
# utils.sepces2df(ces,subsystem)

In [ ]:

fig = viz.plot_ces(subsystem,max_ces[0]['ces'],max_ces[0]['relations'],network_name='figures/space_example',surface_opacity=1.0,)

# 4 nodes toroidal

In [ ]:
# ch3 system creation
net_name = "space_4"

th = 1 / 4
exp = 5

s = 0.8
l = 0.25

weights = np.array(
    [
        [s, l, 0, l],  # A
        [l, s, l, 0],  # B
        [0, l, s, l],  # C
        [l, 0, l, s],  # D
        #A, B, C, D,
    ]
)

# weights += np.random.random((6,6))/100
# weights = np.abs(weights)

node_labels = ["A", "B", "C", "D"]
mech_func = [f for f in "ssss"]

network = ng.get_net(
    mech_func,
    weights,
    exp=exp,
    th=th,
    mu=mu,
    si=si,
    l=ll,
    k=kk,
    x0=x0,
    node_labels=node_labels,
    network_name=net_name,
    pickle_network=True,
)

state = (0, 0, 0, 0)
subsystem = pyphi.Subsystem(network, state)

In [ ]:
ces = utils.flatten([[d.cause,d.effect] for d in pyphi.compute.ces(subsystem,parallel=True)])
all_compositional_states = compute.get_all_compositional_states(ces)
print('there are {} possible compositional states'.format(len(all_compositional_states)))

In [ ]:
max_ces = max(
    [
        compute.filter_ces_parallel(subsystem, ces, compositional_state, max_relations_k=3, n_jobs=200)
        for compositional_state in tqdm(all_compositional_states)
    ],
    key=lambda c: c["big phi"],
)

In [ ]:
max_ces['big phi']

In [ ]:
utils.sepces2df(max_ces['ces'],subsystem)

In [ ]:
max_ces['MIP']

In [ ]:

fig = viz.plot_ces(subsystem,max_ces['ces'],max_ces['relations'],network_name='figures/space',surface_opacity=0.7,)

# 5 nodes linear

In [ ]:
# ch3 system creation
net_name = "space_5"

th = 1 / 4
exp = 5

s = 0.8
l = 0.25
f = 0.06
g = 0.01

weights = np.array(
    [
        [s, l, f, g, 0],  # A
        [l, s, l, f, g],  # B
        [f, l, s, l, f],  # C
        [g, f, l, s, l],  # D
        [0, g, f, l, s],  # E
        #A, B, C, D, E
    ]
)

# weights += np.random.random((6,6))/100
# weights = np.abs(weights)

node_labels = ["A", "B", "C", "D", "E"]
mech_func = [f for f in "sssss"]

network = ng.get_net(
    mech_func,
    weights,
    exp=exp,
    th=th,
    node_labels=node_labels,
    network_name=net_name,
    pickle_network=True,
)

state = (0, 0, 0, 0, 0)
subsystem = pyphi.Subsystem(network, state)

In [ ]:
ces = utils.flatten([[d.cause,d.effect] for d in pyphi.compute.ces(subsystem,parallel=True)])
all_compositional_states = compute.get_all_compositional_states(ces)
print('there are {} possible compositional states'.format(len(all_compositional_states)))

In [ ]:
relations = pyphi.relations.relations(subsystem,ces,parallel=True,max_order=3,parallel_kwargs=dict(verbose=5,n_jobs=-1))

In [ ]:
rels = compute.compute_relations(subsystem, ces, max_k=3, num_relations=False)

In [ ]:
max_ces = compute.filter_ces(subsystem, ces, compute.compositional_state_from_system_state(subsystem.state), max_relations_k=3, n_jobs=-1)

In [ ]:
max_ces['big phi']

In [ ]:
utils.sepces2df(max_ces['ces'],subsystem)

In [ ]:
max_ces['MIP']

In [ ]:

fig = viz.plot_ces(subsystem,max_ces['ces'],max_ces['relations'],network_name='figures/space',surface_opacity=0.7,)

In [ ]:
import itertools
compositional_state = compute.compositional_state_from_system_state(subsystem.state)

# first separate the ces into mices and define the directions
c = pyphi.direction.Direction.CAUSE
e = pyphi.direction.Direction.EFFECT

# next we run through all the mices and append any mice that has a state corresponding to the compositional state
mices_with_correct_state = dict()  # compositional_state.copy()
for mice in ces:
    if (
        tuple(mice.specified_state[0])
        == compositional_state[mice.direction][mice.purview]
    ):
        if not (mice.direction, mice.purview) in mices_with_correct_state.keys():
            mices_with_correct_state[(mice.direction, mice.purview)] = [mice]
        else:
            mices_with_correct_state[(mice.direction, mice.purview)].append(mice)

all_cess = list(itertools.product(*mices_with_correct_state.values()))

In [ ]:
len(all_cess)

In [ ]:
[len(l) for l in mices_with_correct_state.values()]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dists = np.array([[np.random.lognormal(m,s,100000) for m in [0,0.5,1,2]] for s in [0.1, 0.5, 1, 3]])
        

In [ ]:
bins=np.linspace(0,10,100)

In [ ]:
ms = dists.shape[0]
ss = dists.shape[1]

plt.figure(figsize=(15,15))
for i in range(ms):
    for j in range(ss):
        plt.subplot(ss,ms,i+j*ms+1)
        plt.hist(dists[i,j,:],bins=bins)
        plt.xlim([0,10])

In [ ]:
network_generator.NR(1, 5, 0.25)

In [ ]:
exp = 5
th = 0.25

x = np.linspace(0,2,100)
y = np.array([network_generator.NR(xx, exp, th) for xx in x])


In [ ]:
plt.plot(x,y)

In [ ]:

vals = [0.7, 0.25, 0.1]
ps = [network_generator.NR(xx, exp, th) for xx in vals]

In [ ]:
ps